# Data Preprocessing


    In this notebook we filter the data by the selected channels, give the timestamps as an index and annotate the seizure sequences

In [22]:
from source import data_import
from source.constants import CHANNELS, DEFAULT_PATIENTS
import pandas as pd

In [23]:
# p_df = data_import.import_patients(patient_ids=[1, 2, 3, 4], seizure_flag=True)
# p_df.head()

In [24]:
# data_import.save_pyarrow(p_df, file_name="preprocess_test")

In [25]:
# import segmented data previous version (mixed ictal and preictal sequences)

# s_df = data_import.load_segmented_data(patient_ids=[1,2],
#                                         nr_segments=30,
#                                         segment_duration=5,
#                                         ictal_segmentation_foo=data_import.ictal_segmentation,
#                                         interictal_segmentation_foo=data_import.inter_segmentation,
#                                         channels=CHANNELS)

In [26]:
pyarrow_file_name = "segmented_data_300"

In [27]:
# import segmented data with purely preictal and interictal epochs.

s_df = data_import.load_segmented_data(patient_ids=DEFAULT_PATIENTS,
                                        nr_segments=300,
                                        segment_duration=1,
                                        ictal_segmentation_foo=data_import.preictal_segmentation,
                                        interictal_segmentation_foo=data_import.inter_segmentation,
                                        channels=CHANNELS)

chb01_01.edf was import but not resampled 256Hz.
chb01_02.edf was import but not resampled 256Hz.
chb01_03.edf was import but not resampled 256Hz.
chb01_03.edf seizure and buffer was labeled
chb01_04.edf was import but not resampled 256Hz.
chb01_04.edf seizure and buffer was labeled
chb01_05.edf was import but not resampled 256Hz.
chb01_06.edf was import but not resampled 256Hz.
chb01_07.edf was import but not resampled 256Hz.
chb01_08.edf was import but not resampled 256Hz.
chb01_09.edf was import but not resampled 256Hz.
chb01_10.edf was import but not resampled 256Hz.
chb01_11.edf was import but not resampled 256Hz.
chb01_12.edf was import but not resampled 256Hz.
chb01_13.edf was import but not resampled 256Hz.
chb01_14.edf was import but not resampled 256Hz.
chb01_15.edf was import but not resampled 256Hz.
chb01_15.edf seizure and buffer was labeled
chb01_16.edf was import but not resampled 256Hz.
chb01_16.edf seizure and buffer was labeled
chb01_17.edf was import but not resample

In [28]:
data_import.save_pyarrow(s_df, file_name=pyarrow_file_name)

/home/weasel/reps/ai-seizure-detectives/source/../data/segmented_data_300.arrow was successfully written.


In [29]:
# p_df = data_import.load_pyarrow(file_name="preprocess_test")
p_df = data_import.load_pyarrow(file_name=pyarrow_file_name)

/home/weasel/reps/ai-seizure-detectives/source/../data/segmented_data_300.arrow was loaded.


In [30]:
p_df.isna().sum()

channel
F7-T7             0
FP1-F3            0
F3-C3             0
FP2-F4            0
F4-C4             0
T8-P8-0           0
FZ-CZ             0
CZ-PZ             0
FT9-FT10          0
T8-P8-1           0
is_seizure        0
before_seizure    0
file              0
seizure_start     0
target            0
epoch             0
segment_id        0
dtype: int64

In [31]:
p_df.shape

(33561600, 17)

In [32]:
from source.filter_eeg_channels import filter_eeg_channels

exclude_ranges=[[58, 62], [118, 122]]
fit_df = filter_eeg_channels(p_df, CHANNELS, fs=256, exclude_ranges=exclude_ranges, Q=30)
fit_df.head()

: 

In [ ]:
fit_df.isna().sum()

channel
F4-C4             0
F3-C3             0
FT9-FT10          0
FZ-CZ             0
F7-T7             0
FP2-F4            0
T8-P8-1           0
T8-P8-0           0
FP1-F3            0
CZ-PZ             0
before_seizure    0
is_seizure        0
target            0
dtype: int64

In [ ]:
pd_toconcat = p_df[['epoch', 'segment_id']]
pd_toconcat.shape

(16857600, 2)

In [ ]:
fit_df = pd.concat(objs=[fit_df, pd_toconcat], axis =1)
fit_df.head()

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure,target,epoch,segment_id
0 days 00:30:00,16.878726,20.334213,-3.322584,17.941953,18.739373,33.092935,12.625819,12.625819,44.788430,2.790970,False,False,0 days,0,0
0 days 00:30:00.003906250,22.891349,24.865283,-8.175141,23.784967,26.448730,40.639022,11.874291,11.874291,59.506243,4.538652,False,False,0 days,0,0
0 days 00:30:00.007812500,22.162881,25.360125,-8.902114,25.615925,27.023819,40.074853,5.285063,5.285063,60.136641,5.292160,False,False,0 days,0,0
0 days 00:30:00.011718750,29.573426,33.953147,-14.930178,33.038895,32.700204,52.956037,8.308854,8.308854,79.842157,8.558853,False,False,0 days,0,0
0 days 00:30:00.015625,21.292400,18.163275,-11.955401,21.366748,20.938086,35.328903,-1.132117,-1.132117,60.607725,6.096930,False,False,0 days,0,0


In [ ]:
### go through non-seizure data (before_seizure)
# extract seizure-starts
    # for each seizure start
        # get previous 20 seconds
        # check if is_seizure in it AND if all are before_seizure
            # yes: drop and continue to next seizure start
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

### go through seizure data (is_seizure)
# take seizre-starts
    # for each seizure start
        # get next 20 seconds
        # check if all are is_seizure
            # no: drop and continue
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

# out put: data resampled to 20sec with features

In [ ]:
fit_df.info()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 16857600 entries, 0 days 00:30:00 to 0 days 00:32:29.996093750
Data columns (total 15 columns):
 #   Column          Dtype          
---  ------          -----          
 0   F4-C4           float64        
 1   F3-C3           float64        
 2   FT9-FT10        float64        
 3   FZ-CZ           float64        
 4   F7-T7           float64        
 5   FP2-F4          float64        
 6   T8-P8-1         float64        
 7   T8-P8-0         float64        
 8   FP1-F3          float64        
 9   CZ-PZ           float64        
 10  before_seizure  bool           
 11  is_seizure      bool           
 12  target          timedelta64[ns]
 13  epoch           int64          
 14  segment_id      int64          
dtypes: bool(2), float64(10), int64(2), timedelta64[ns](1)
memory usage: 1.8 GB


In [ ]:
fit_df['seizure_start'] = fit_df['is_seizure'] & ~fit_df['is_seizure'].shift(fill_value=False)
fit_df.loc[fit_df['seizure_start'] == True, 'seizure_start']

Series([], Name: seizure_start, dtype: bool)

In [ ]:
fit_df['seizure_id'] = fit_df['seizure_start'].cumsum()
fit_df['seizure_id'].tail()

0 days 00:32:29.980468750    0
0 days 00:32:29.984375       0
0 days 00:32:29.988281250    0
0 days 00:32:29.992187500    0
0 days 00:32:29.996093750    0
Name: seizure_id, dtype: int64

In [ ]:
fit_df.head()

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure,target,epoch,segment_id,seizure_start,seizure_id
0 days 00:30:00,16.878726,20.334213,-3.322584,17.941953,18.739373,33.092935,12.625819,12.625819,44.788430,2.790970,False,False,0 days,0,0,False,0
0 days 00:30:00.003906250,22.891349,24.865283,-8.175141,23.784967,26.448730,40.639022,11.874291,11.874291,59.506243,4.538652,False,False,0 days,0,0,False,0
0 days 00:30:00.007812500,22.162881,25.360125,-8.902114,25.615925,27.023819,40.074853,5.285063,5.285063,60.136641,5.292160,False,False,0 days,0,0,False,0
0 days 00:30:00.011718750,29.573426,33.953147,-14.930178,33.038895,32.700204,52.956037,8.308854,8.308854,79.842157,8.558853,False,False,0 days,0,0,False,0
0 days 00:30:00.015625,21.292400,18.163275,-11.955401,21.366748,20.938086,35.328903,-1.132117,-1.132117,60.607725,6.096930,False,False,0 days,0,0,False,0


In [ ]:
fit_df.isna().sum()

channel
F4-C4             0
F3-C3             0
FT9-FT10          0
FZ-CZ             0
F7-T7             0
FP2-F4            0
T8-P8-1           0
T8-P8-0           0
FP1-F3            0
CZ-PZ             0
before_seizure    0
is_seizure        0
target            0
epoch             0
segment_id        0
seizure_start     0
seizure_id        0
dtype: int64

In [ ]:
data_import.save_pyarrow(fit_df, file_name=pyarrow_file_name)

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocessed_seg_all.arrow was successfully written.
